<a href="https://colab.research.google.com/github/ouriquco/Classificiation-Academic-Success/blob/summer_models/Academic_Success.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [10]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

# Reading in Data

In [3]:
train = pd.read_csv('train.csv', index_col = 'id')
test = pd.read_csv('test.csv', index_col = 'id')

# Data Preprocessing

In [11]:
#checking for NA
train.isnull().sum()
test.isnull().sum()

#checking for duplicates
train.duplicated().sum()
test.duplicated().sum()

features = test.columns

#convert categorical data to numeric
if train['Target'].dtype == 'object':
    le = LabelEncoder()
    train['Target'] = le.fit_transform(train['Target'])

#splitting into features and target
X_train = train[features]
y_train = train['Target']
X_test = test[features]

# Data Visualization

# Cross Validation

# Random Forest

In [13]:
#initialize Random Forest - play with inputs here
rf = RandomForestClassifier()

# Fit the model on the training data
rf.fit(X_train, y_train)

# Predict on the test data
y_pred = rf.predict(X_test)

#Evaluation

#Accuracy
print("Accuracy:", accuracy_score(y_train, rf.predict(X_train)))

#Confusion Matrix
print("Confusion Matrix:\n", confusion_matrix(y_train, rf.predict(X_train)))

#Precision
print("Precision:", precision_score(y_train, rf.predict(X_train), average = 'macro'))

#Recall
print("Recall:", recall_score(y_train, rf.predict(X_train), average = 'macro'))

#F1 Score
print("F1 Score:", f1_score(y_train, rf.predict(X_train), average = 'macro'))

#cross validation
cv_scores = cross_val_score(rf, X_train, y_train, cv=5)
print("Cross-Validation Scores:", cv_scores)
print("Average Cross-Validation Score:", cv_scores.mean())

#create submission file
#convert target back from numbers into "graduate", "dropout", or "enrolled"
labels = le.inverse_transform(y_pred)

submission = pd.DataFrame({'id': test.index, 'Target': labels})
submission.to_csv('submission.csv', index=False)


Accuracy: 0.9999869311795917
Confusion Matrix:
 [[25296     0     0]
 [    1 14939     0]
 [    0     0 36282]]
Precision: 0.9999868232069679
Recall: 0.9999776885319053
F1 Score: 0.9999822553658495
Cross-Validation Scores: [0.82631992 0.82462101 0.82697334 0.82735411 0.82068875]
Average Cross-Validation Score: 0.8251914276381


# AdaBoost

# Gradient Boosting

# XGBoost

## CatBoost

# LightGBM